# 生成器

本章中我们将深入了解 Python *生成器 (generator)*，包括*生成器表达式 (generator expressions)* 和 *生成器函数 (generator functions)*。

## 生成器表达式

列表推导和生成器表达式的区别有的时候令人十分困惑。这里我们快速领略一下它们的区别：

### 列表推导使用方括号，而生成器表达式使用圆括号

这是一个有代表性的列表推导：

In [1]:
[n ** 2 for n in range(12)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]

而这是一个有代表性的生成器表达式：

In [2]:
(n ** 2 for n in range(12))

<generator object <genexpr> at 0x10fe0dfc0>

注意直接打印生成器表达式并不会打印出内容。要打印出生成器表达式生成的全部内容的一个方法是通过向 ``list`` 构造器传递参数：

In [3]:
G = (n ** 2 for n in range(12))
list(G)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]

### 列表是数值的集合，而生成器是生成数值的方法

当你创建一个列表的时候，你**实际上**在产生一个数值的集合，并且需要花费一定的内存成本。当你创建一个生成器的时候，你并没有创建一个数值的集合，而是建立了一个产生这些数值的方法。这两者都具有相同的迭代器接口，正如我们在这里看到的：

In [4]:
L = [n ** 2 for n in range(12)]
for val in L:
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 100 121 

In [5]:
G = (n ** 2 for n in range(12))
for val in G:
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 100 121 

区别是，生成器表达式并不会在数值被需要之前计算出它们。这并不仅仅可以提高内存效率，也能提高计算效率！这也意味着，一个列表的大小受可用内存限制时，而生成器表达式的大小是无限的！

我们可以通过 ``itertools`` 中定义的 ``count`` 迭代器创建一个无限的产生器表达式的例子：

In [6]:
from itertools import count
count()

count(0)

In [7]:
for i in count():
    print(i, end=' ')
    if i >= 10: break

0 1 2 3 4 5 6 7 8 9 10 

``count`` 迭代器会开心地计数直到进程被手动中断或结束。这使得创建一个永远不会停止的生成器非常方便：

In [8]:
factors = [2, 3, 5, 7]
G = (i for i in count() if all(i % n > 0 for n in factors))
for val in G:
    print(val, end=' ')
    if val > 40: break

1 11 13 17 19 23 29 31 37 41 

你可能看到我们在这里得到什么：如果我们把这个包含因子的列表合适地展开的话，我们在开头会得到一个素数的产生器（通过*埃拉托色尼筛选法 (the Sieve of Eratosthenes algorithm)*）。我们马上就会开始继续研究产生素数这个话题。

### 列表可以被循环迭代多次，而一个生成器表达式只能一次性使用

这是生成器表达式潜在的问题之一。对于一个列表，我们可以直接对它做：

In [9]:
L = [n ** 2 for n in range(12)]
for val in L:
    print(val, end=' ')
print()

for val in L:
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 100 121 
0 1 4 9 16 25 36 49 64 81 100 121 

另一方面，生成器表达式在一次迭代后就被用尽：

In [10]:
G = (n ** 2 for n in range(12))
list(G)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]

In [11]:
list(G)

[]

这个特性可以变得非常实用，因为这意味着迭代可以被停止和继续：

In [12]:
G = (n**2 for n in range(12))
for n in G:
    print(n, end=' ')
    if n > 30: break

print("\ndoing something in between")

for n in G:
    print(n, end=' ')

0 1 4 9 16 25 36 
doing something in between
49 64 81 100 121 

一个我发现这个特性非常实用的场合是在对磁盘上的数据文件集合进行处理时，这个特性意味着你可以非常轻松地批量处理数据，让生成器负责跟踪那些你尚未处理的块。

## 生成器函数：使用关键字 ``yield``

在之前的章节中我们看到了列表推导是创建相对简单的列表的最佳途径，而常规的 ``for`` 循环在更加复杂的情况下更为合适。对于生成器表达式来说这也是成立的：我们可以通过*生成器函数 (generator function)* 使用 ``yield`` 语句来创建更加复杂的生成器。

在这里我们有两种方式构造同一个列表：

In [13]:
L1 = [n ** 2 for n in range(12)]

L2 = []
for n in range(12):
    L2.append(n ** 2)

print(L1)
print(L2)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]


类似地，我们有两种方法构造等价的生成器：

In [14]:
G1 = (n ** 2 for n in range(12))

def gen():
    for n in range(12):
        yield n ** 2

G2 = gen()
print(*G1)
print(*G2)

0 1 4 9 16 25 36 49 64 81 100 121
0 1 4 9 16 25 36 49 64 81 100 121


一个生成器函数是这样的一个函数：它不使用 ``return`` 返回一个值（仅返回一次），而是使用 ``yield`` 来产生一个包含（可能是无穷多的）数值的序列。和生成器表达式中一样，在部分迭代的间隔中（译者注：两次迭代的中间，此时生成器并没有被从头到尾执行完，故称部分迭代）生成器的状态得到保留。但是如果我们需要生成器的一个全新的副本，我们只需要简单地重新调用那个函数即可。

## 样例：素数生成器

这里我将展示我最喜欢的一个生成器函数的例子：一个函数来产生无限的素数序列。解决这个问题的一个经典算法是埃拉托色尼筛选法，它的工作过程类似下面的代码：

In [15]:
# 生成一个包含候选数值的列表
L = [n for n in range(2, 40)]
print(L)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


In [16]:
# 对于候选数值序列的第一个值，移除它的倍数
L = [n for n in L if n == L[0] or n % L[0] > 0]
print(L)

[2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39]


In [17]:
# 对于候选数值序列的第二个值，移除它的倍数
L = [n for n in L if n == L[1] or n % L[1] > 0]
print(L)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 25, 29, 31, 35, 37]


In [18]:
# 对于候选数值序列的第三个值，移除它的倍数
L = [n for n in L if n == L[2] or n % L[2] > 0]
print(L)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37]


如果我们不断在一个足够大的列表上重复这个过程足够多次，我们就能产生足够多我们需要的素数。

让我们把这个计算的逻辑封装成为一个生成器表达式：

In [19]:
def gen_primes(N):
    """Generate primes up to N"""
    primes = set()
    for n in range(2, N):
        if all(n % p > 0 for p in primes):
            primes.add(n)
            yield n

print(*gen_primes(100))

2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97


这就是关于生成器的一切！尽管这绝对不是高效实现埃拉托色尼筛选法的方法，但它说明了如何使用生成器函数的语法来方便地构建更复杂的序列。